In [1]:
import os, shutil, json, codecs
import psycopg2
import pandas as pd
import matplotlib as plt
import numpy as np

In [2]:
file = "Student @Information!.csv";
df = pd.read_csv(file)
df

,ID,First Name,Last Name,Gender,Username,Email,University,Language
0,1,Cecilius,Jewsbury,Male,cjewsbury0,cjewsbury0@pen.io,"Swinburne University of Technology, Sarawak Ca...",Bengali
1,2,Garik,Lawford,Male,glawford1,glawford1@trellian.com,Bangladesh University of Professionals,Bislama
2,3,Estrellita,Azam,Female,eazam2,eazam2@ustream.tv,University of Peradeniya,Tok Pisin
3,4,Christabel,de Keep,Female,cdekeep3,cdekeep3@census.gov,Mississippi State University,Hebrew
4,5,Ev,Darrel,Male,edarrel4,edarrel4@examiner.com,Southern Connecticut State University,Aymara
...,...,...,...,...,...,...,...,...
995,996,Hillel,Gunson,Male,hgunsonrn,hgunsonrn@example.com,Ecole des Hautes Etudes Commerciales MAROC,Gujarati
996,997,Calv,Antonutti,Male,cantonuttiro,cantonuttiro@123-reg.co.uk,North Eastern Hill University,Lithuanian
997,998,Chip,Hanney,Male,channeyrp,channeyrp@kickstarter.com,Mordovian State University,Belarusian
998,999,Ric,Linde,Male,rlinderq,rlinderq@imgur.com,Université Kongo,Punjabi


In [3]:
# adjust table name
file_name = file.split(".")[0]
table_name = file_name.lower().replace(" ", "_").replace("!", "").replace("@", "")
table_name

'student_information'

In [4]:
df.columns

Index(['ID', 'First Name', 'Last Name', 'Gender', 'Username', 'Email',
       'University', 'Language'],
      dtype='object')

In [5]:
df.columns = [x.lower().replace(" ", "_") for x in df.columns]
df.columns

Index(['id', 'first_name', 'last_name', 'gender', 'username', 'email',
       'university', 'language'],
      dtype='object')

In [6]:
conn = cur = None
conn = psycopg2.connect(
    host="localhost", 
    dbname="csv_db", 
    user="postgres", 
    password="namvt16052004", 
    port=5432
)
cur = conn.cursor()
if (conn and cur):
    print("Connect successfully!")

Connect successfully!


In [7]:
# create table student_information
# (
#     id                int,
#     first_name        varchar,
#     last_name         varchar,
#     gender            varchar,
#     username          varchar,
#     email             varchar,
#     university        varchar,
#     language          varchar
# );

In [8]:
df.dtypes

id             int64
first_name    object
last_name     object
gender        object
username      object
email         object
university    object
language      object
dtype: object

In [9]:
replacements = {
    "object": "varchar",
    "int64": "int",
    "float64": "float",
    "datetime64": "timestamp",
    "timedelta64[ns]": "varchar"
}

replacements

{'object': 'varchar',
 'int64': 'int',
 'float64': 'float',
 'datetime64': 'timestamp',
 'timedelta64[ns]': 'varchar'}

In [10]:
col_str = ", ".join("{} {}".format(n, d) for (n, d) in zip(df.columns, df.dtypes.replace(replacements)))
col_str

'id int, first_name varchar, last_name varchar, gender varchar, username varchar, email varchar, university varchar, language varchar'

In [11]:
# drop table if exists
command = f"drop table if exists {table_name}"
cur.execute(command)

In [12]:
# create tables
command = f"create table {table_name} (id int, first_name varchar, last_name varchar, gender varchar, username varchar, email varchar, university varchar, language varchar)"
cur.execute(command)

In [13]:
# new csv
df.to_csv((table_name + ".csv"), header=df.columns, index=False, encoding="utf-8")
df

,id,first_name,last_name,gender,username,email,university,language
0,1,Cecilius,Jewsbury,Male,cjewsbury0,cjewsbury0@pen.io,"Swinburne University of Technology, Sarawak Ca...",Bengali
1,2,Garik,Lawford,Male,glawford1,glawford1@trellian.com,Bangladesh University of Professionals,Bislama
2,3,Estrellita,Azam,Female,eazam2,eazam2@ustream.tv,University of Peradeniya,Tok Pisin
3,4,Christabel,de Keep,Female,cdekeep3,cdekeep3@census.gov,Mississippi State University,Hebrew
4,5,Ev,Darrel,Male,edarrel4,edarrel4@examiner.com,Southern Connecticut State University,Aymara
...,...,...,...,...,...,...,...,...
995,996,Hillel,Gunson,Male,hgunsonrn,hgunsonrn@example.com,Ecole des Hautes Etudes Commerciales MAROC,Gujarati
996,997,Calv,Antonutti,Male,cantonuttiro,cantonuttiro@123-reg.co.uk,North Eastern Hill University,Lithuanian
997,998,Chip,Hanney,Male,channeyrp,channeyrp@kickstarter.com,Mordovian State University,Belarusian
998,999,Ric,Linde,Male,rlinderq,rlinderq@imgur.com,Université Kongo,Punjabi


In [14]:
# open csv file
myFile = open(table_name + ".csv")
if (myFile):
    print("Open successfully")

Open successfully


In [15]:
# insert
command = "COPY student_information FROM STDIN WITH CSV HEADER DELIMITER AS ','"
cur.copy_expert(sql=command, file=myFile)

In [16]:
# change mode
command = f"grant select on table {table_name} to public"

In [57]:
# close
conn.commit()
if cur is not None:
    cur.close()
if conn is not None:
    conn.close() 